In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [40]:
team_map_schedule = {
    'L.A. Lakers' : 'Los Angeles Lakers',
    'L.A. Clippers' : 'Los Angeles Clippers',
    'Portland' : 'Portland Trailblazers',
    'Cleveland' : 'Cleveland Cavaliers',
    'Denver' : 'Denver Nuggets',
    'Dallas' : 'Dallas Mavericks',
    'Utah' : 'Utah Jazz',
    'San Antonio' : 'San Antonio Spurs',
    'Atlanta' : 'Atlanta Hawks',
    'Charlotte' : 'Charlotte Hornets',
    'Chicago' : 'Chicago Bulls',
    'Detroit' : 'Detroit Pistons',
    'Milwaukee' : 'Milwaukee Bucks',
    'Orlando' : 'Orlando Magic',
    'Minnesota' : 'Minnesota Timberwolves',
    'Phoenix' : 'Phoenix Suns',
    'Boston' : 'Boston Celtics',
    'Indiana' : 'Indiana Pacers',
    'Sacramento' : 'Sacramento Kings',
    'Toronto' : 'Toronto Raptors',
    'Washington' : 'Washington Wizards',
    'Brooklyn' : 'Brooklyn Nets',
    'New Orleans' : 'New Orleans Pelicans',
    'Dallas' : 'Dallas Mavericks',
    'Philadelphia' : 'Philadelphia 76ers',
    'Miami' : 'Miami Heat',
    'Memphis' : 'Memphis Grizzlies',
    'Golden St.' : 'Golden State Warriors',
    'New York' : 'New York Knicks',
    'Houston' : 'Houston Rockets',
    'Oklahoma City' : 'Oklahoma City Thunder'
}

In [72]:
# Instantiating selenium driver
link = 'https://www.cbssports.com/nba/schedule/'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)

# Getting table for today's games and formatting
html = driver.page_source
tables = pd.read_html(html)
today_schedule = tables[1]
today_schedule = today_schedule[['Away', 'Home']]
today_schedule['Away'] = today_schedule.Away.apply(lambda x: team_map_schedule[x])
today_schedule['Home'] = today_schedule.Home.apply(lambda x: team_map_schedule[x])

# Getting table for yesterday's games and formatting
day_buttons = driver.find_elements_by_class_name("ToggleContainer-text")
yesterday = day_buttons[0]
yesterday.click()
html_yesterday = driver.page_source
tables_yesterday = pd.read_html(html_yesterday)
yesterday_schedule = tables_yesterday[1]
yesterday_schedule = yesterday_schedule[['Away', 'Home']]
try:
    yesterday_schedule['Away'] = yesterday_schedule.Away.apply(lambda x: team_map_schedule[x])
    yesterday_schedule['Home'] = yesterday_schedule.Home.apply(lambda x: team_map_schedule[x])
except:
    pass

# Getting table for tomorrow's games and formatting
day_buttons = driver.find_elements_by_class_name("ToggleContainer-text")
today = day_buttons[2]
today.click()
day_buttons = driver.find_elements_by_class_name("ToggleContainer-text")
tomorrow = day_buttons[2]
tomorrow.click()
html_tomorrow = driver.page_source
tables_tomorrow = pd.read_html(html_tomorrow)
tomorrow_schedule = tables_tomorrow[1]
tomorrow_schedule = tomorrow_schedule[['Away', 'Home']]
try:
    tomorrow_schedule['Away'] = tomorrow_schedule.Away.apply(lambda x: team_map_schedule[x])
    tomorrow_schedule['Home'] = tomorrow_schedule.Home.apply(lambda x: team_map_schedule[x])
except:
    pass



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [73]:
today_schedule['is_B2B_First_Away'] = today_schedule.Away.apply(lambda x: 1 if x in 
                                                                list(tomorrow_schedule.Away.unique()) + list(tomorrow_schedule.Home.unique())
                                                               else 0)
today_schedule['is_B2B_First_Home'] = today_schedule.Home.apply(lambda x: 1 if x in 
                                                                list(tomorrow_schedule.Away.unique()) + list(tomorrow_schedule.Home.unique())
                                                               else 0)
today_schedule['is_B2B_Second_Away'] = today_schedule.Away.apply(lambda x: 1 if x in 
                                                                list(yesterday_schedule.Away.unique()) + list(yesterday_schedule.Home.unique())
                                                               else 0)
today_schedule['is_B2B_Second_Home'] = today_schedule.Home.apply(lambda x: 1 if x in 
                                                                list(yesterday_schedule.Away.unique()) + list(yesterday_schedule.Home.unique())
                                                               else 0)

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [74]:
today_schedule

,Away,Home,is_B2B_First_Away,is_B2B_First_Home,is_B2B_Second_Away,is_B2B_Second_Home
0,Los Angeles Lakers,Atlanta Hawks,0,0,0,0
1,Los Angeles Clippers,Charlotte Hornets,0,0,0,0
2,Portland Trailblazers,Chicago Bulls,0,0,0,0
3,Cleveland Cavaliers,Detroit Pistons,0,0,0,0
4,Denver Nuggets,Milwaukee Bucks,0,0,0,0
5,Dallas Mavericks,Orlando Magic,1,0,1,0
6,Utah Jazz,Minnesota Timberwolves,0,0,0,0
7,San Antonio Spurs,Phoenix Suns,0,0,0,0


In [80]:
yesterday_schedule

,Away,Home
0,Boston Celtics,New Orleans Pelicans
1,Indiana Pacers,Dallas Mavericks
2,Sacramento Kings,Philadelphia 76ers
3,Toronto Raptors,Miami Heat
4,Washington Wizards,Memphis Grizzlies
5,Brooklyn Nets,Golden State Warriors
